In [87]:
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline

class neuroLayer:
    def __init__(self, nodes, prevNodes):
        self.layer = numpy.random.normal(0.0, pow(nodes, -0.5), (nodes, prevNodes))
    
    def transposed(self):
        return self.layer.T
    
    def value(self):
        return self.layer

    def setLayer(value):
        self.layer = value
    
    
def defaultNet():
    return neuroNet(3, 3, 3, 0.3)

def numberNet():
    return neuroNet(784, 100, 10, 0.3)

def readFromFile(path):
    toread = open(path, "r")
    ret = toread.readlines()
    toread.close()
    return ret

class neuroNet:
    def __init__(self, iNodes, hNodes, oNodes, learnRate, additional_layers = 0):
        self.lr = learnRate
        self.layers = []
        self.activation_function = lambda x: scipy.special.expit(x)
        
        self.layers.append(neuroLayer(hNodes, iNodes))

        for i in range(0, additional_layers):
            self.layers.append(neuroLayer(hNodes, hNodes))
        
        self.layers.append(neuroLayer(oNodes, hNodes))
        
    def updateNet(self, errors, outputs, inputs):
         return self.lr * numpy.dot((errors * outputs * (1.0 - outputs)), numpy.transpose(inputs))
        
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        Inputs = []
        Outputs = []
        Errors = []
        last = -1
        
        inp = inputs
        
        #inputs/outputs per layer
        for layer in self.layers:
            if len(Inputs) == 0:
                Inputs.append(inp) 
                inp = numpy.dot(layer.value(), inp)  
            else:
                Inputs.append(Outputs[last])
                inp = numpy.dot(layer.value(), Outputs[last])
                
            Outputs.append(self.activation_function(inp))
            last += 1
            
        #errors per layer (reverse order)
        lastLayer = None
        for layer in self.layers[::-1]:
            if len(Errors) == 0:
                Errors.append(targets - Outputs[last])
            else:
                Errors.insert(0, numpy.dot(lastLayer.layer.T, Errors[0]))
            lastLayer = layer
        
        first = True
        idx = 0
        #update values per layer
        for layer in self.layers:
            layer.layer += self.updateNet(Errors[idx], Outputs[idx], Inputs[idx])
            idx += 1

    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        for layer in self.layers:
            inputs = self.activation_function(numpy.dot(layer.value(), inputs))  
        return inputs
    
# main()
#n = numberNet()
#n.query([1.0, 0.5, -1.5])  
print("k")

k


In [90]:
def trainNumberNet(net, data):
    for record in data:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        net.train(inputs, targets)

input_nodes = 784
hidden_nodes = 80
output_nodes = 10
learning_rate = 0.25
additional_layers = 1

n = neuroNet(input_nodes, hidden_nodes, output_nodes, learning_rate, additional_layers)

training_data_list = readFromFile("../dat/mnist_train.csv")

trainNumberNet(n, training_data_list)
print("trained")

trained


In [91]:
#test net
test_data_list = readFromFile("../dat/mnist_test.csv")
scorecard = []
for record in test_data_list:
    all_values = record.split(",")
    correct_label = int(all_values[0])
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.query(inputs)
    label = numpy.argmax(outputs)
    if label == correct_label:
        scorecard.append(1)
    else:
        scorecard.append(0)
scorecard_array = numpy.asarray(scorecard)
print("correct: ", scorecard_array.sum() / scorecard_array.size * 100, "%")

correct:  81.19 %
